# Uma análise quantitativa dos hotéis em Recife a partir dos dados do Trivago

Ângelo de Sant'Ana &mdash;
assd {em} cin.ufpe.br;

Antônio Rodrigues &mdash;
acpr {em} cin.ufpe.br;

Juliana Carvalho &mdash;
jco2 {em} cin.ufpe.br

junho de 2017

In [49]:
from bs4 import BeautifulSoup
import pandas as pd
import re

##### Localização dos dados relevantes

Abaixo começamos a localização dos elementos importantes das páginas .html para reconhecimento dos conjuntos que contêm
os valores que desejamos usar na nossa análise.

##### Dados sobre avaliação dos hotéis

In [50]:
# fazer um iterador para capturar todas as páginas de 1 a 5

pdGeralDataScraping = pd.DataFrame

pdAppended = pd.DataFrame
i = 1

while i <= 5:
    
    urlAvaliacoes = "pages/avaliacoesPag%d.html" % i

    parsed_page_avaliacoes = BeautifulSoup(open(urlAvaliacoes,encoding="utf-8"),"html.parser")

    listaHoteis = []

    nomesDosHoteisAvaliacoes = parsed_page_avaliacoes.find_all('h3', class_="name__copytext m-0 item__slideout-toggle")

    divsAvaliacoes = parsed_page_avaliacoes.find_all('div', class_="rat-chart__group rat-chart__group--collapsed")

    avaliacaoToInt = lambda x: int(re.sub('[\/][0-9]{3}', '', re.sub('[a-zA-ZzáàâãéèêíïóôõöúçñÁÀÂÃÉÈÍÏÓÔÕÖÚÇÑ\s]+','',x)))

    aggregate = list(zip(nomesDosHoteisAvaliacoes, divsAvaliacoes))

    for (hotelName, avaliacoes) in aggregate:
        valoresAvaliacoes = avaliacoes.find_all(class_="rat-chart__number");

        listaValoresAux = []
        listaValoresAux.append(hotelName.text)

        for valor in valoresAvaliacoes:
            if "Não" in valor.text:
                listaValoresAux.append(valor.text)
            else:
                listaValoresAux.append(int(avaliacaoToInt(valor.text)))

    #         print(listaValoresAux)

        listaHoteis.append(dict(zip(["Nome", "localizacao", "quartos", "servico", "limpeza", "custo-beneficio", \
                                     "conforto", "facilidades", "predio", "cafe_da_manha", "comida"], listaValoresAux)))

    # print(listaHoteis)

        temp = pd.DataFrame(listaHoteis, columns=["Nome", "localizacao", "quartos", "servico", "limpeza", \
                                          "custo-beneficio", "conforto", "facilidades", "predio", "cafe_da_manha", "comida"])
    if i == 1:
        pdAppended = temp
    else:
        pdAppended = pd.concat([pdAppended,temp],ignore_index=True)
        
    i += 1
    
pdGeralDataScraping = pdAppended
pdGeralDataScraping
# pdAppended.to_csv('saidaDataScrapingAval.csv',sep=';',encoding='utf-8')


,Nome,localizacao,quartos,servico,limpeza,custo-beneficio,conforto,facilidades,predio,cafe_da_manha,comida
0,Radisson Recife,92,84,79,85.0,78,85,83,80,Não avaliadoNão disponível,76
1,Golden Tulip Recife Boa Viagem,89,80,84,81.0,76,81,80,75,Não avaliadoNão disponível,77
2,Mar Hotel Conventions,86,82,79,84.0,77,83,83,79,70,75
3,Best Western Hotel Manibu Recife,84,77,82,86.0,77,82,82,82,75,75
4,Transamérica Prestige Beach Class International,89,84,88,88.0,79,89,85,85,Não avaliadoNão disponível,Não avaliadoNão disponível
5,Beach Class Conselheiro By Nobile,Não avaliadoNão disponível,Não avaliadoNão disponível,83,87.0,Não avaliadoNão disponível,83,Não avaliadoNão disponível,81,Não avaliadoNão disponível,Não avaliadoNão disponível
6,Hotel Atlante Plaza,91,81,84,85.0,77,84,81,78,78,76
7,Nobile Beach Class Executive,88,83,84,87.0,79,86,84,81,72,80
8,Hotel Duble,84,72,78,77.0,75,72,78,70,Não avaliadoNão disponível,78
9,Parkhotel,89,76,75,81.0,75,77,79,72,78,80


#### Data Scraping das Páginas de Informação dos Hotéis

In [51]:
pdAppended = pd.DataFrame

cont = 1

while cont <= 5:

    urlInfos = "pages/InfoPag%d.html" % cont

    parsed_page_infos = BeautifulSoup(open(urlInfos,encoding="utf-8"),"html.parser")

    # pegando os nomes do hotéis
    nomesDosHoteisInfos = parsed_page_infos.find_all('h3', class_="name__copytext m-0 item__slideout-toggle")

    # pegando o conjunto de comodidades de cada hotel
    divsInfos = parsed_page_infos.find_all('div', class_="sl-box__block clearfix")

    aggregate = list(zip(nomesDosHoteisInfos, divsInfos))

    #Lista com os nomes dos hotéis encontrados da pagina
    listaNomesHotel = []
    #Lista de listas das comodidades dos hotéis
    listaGeralComodidadesQuarto = []
    #Lista de listas das comodidades dos quartos dos hotéis
    listaGeralComodidadesHotel = []
    
    #Pegando as comodidades dos hotéis e dos quartos e salvando nas listas acima
    for (hotelName, divInfo) in aggregate:

        listaNomesHotel.append(hotelName.text)

        uls = divInfo.find_all('div',class_="all-amenities__group mb-gutter")
        listaComodidadesHotel = uls[0].find_all('li')
        listaComodidadesQuarto = uls[1].find_all('li')

        listaComodHotel = []

        for comodsHotel in listaComodidadesHotel:
            listaComodHotel.append(comodsHotel.text)

        listaComodQuarto = []

        for comodsQuarto in listaComodidadesQuarto:
            listaComodQuarto.append(comodsQuarto.text)

        listaGeralComodidadesHotel.append(listaComodHotel)
        listaGeralComodidadesQuarto.append(listaComodQuarto)
    
    #Listas com todas as possíveis comodidades
    comodidadesHotel = ['Academia','Acesso direto à praia','Animais permitidos','Bar na beira da Piscina','Bar no hotel','Cabeleireiro','Centro financeiro','Check-in / out Expresso','Cofre na recepção','Concierge','Elevador','Estacionamento','Guarda-sol','Lavanderia','Loja de conveniência','PC com Internet','Piscina exterior','Piscina interna','Quarto para não-fumantes','Recepção 24 horas','Restaurante','Sala de conferências','Serviço de portaria','Serviço de quarto','Serviço de quarto 24 horas','Spa','Terraço','Transfer de/ para aeroporto','Wi-Fi grátis no lobby']
    comodidadesQuarto = ['Acesso Wi-Fi nos quartos','Ar condicionado','Banheiro com banheira','Banheiro com ducha','Canais de TV via satélite','Cofre no quarto','Cozinha quitinete','Espelho de aumento','Ferro e tábua de passar','Geladeira','Internet','Mesa de trabalho','Microondas','Secador','Telefone','TV a cabo','TV LCD','Varanda / Terraço','Wi-Fi grátis nos quartos']

    #Listas que serão preenchidas com listas de 0s e/ou 1s caso o hotel iterado possua a comodidade em questão
    listasBinariasAtributosHoteis = []
    listasBinariasAtributosQuartos = []

    #Preenchendo com 0 ou 1 as listas, comparando a lista de todas as comodidades com a de cada hotel
    for listaComodidadeDoHotel in listaGeralComodidadesHotel:
        listaBinariaAtributosHotel = []

        for comodidade in comodidadesHotel:
            if comodidade in listaComodidadeDoHotel:
                listaBinariaAtributosHotel.append('1')
            else:
                listaBinariaAtributosHotel.append('0')

        listasBinariasAtributosHoteis.append(listaBinariaAtributosHotel)

    #Preenchendo com 0 ou 1 as listas, comparando a lista de todas as comodidades com a de cada hotel
    for listaComodidadeDoQuarto in listaGeralComodidadesQuarto:
        listaBinariaAtributosQuarto = []

        for comodidade in comodidadesQuarto:
            if comodidade in listaComodidadeDoQuarto:
                listaBinariaAtributosQuarto.append('1')
            else:
                listaBinariaAtributosQuarto.append('0')

        listasBinariasAtributosQuartos.append(listaBinariaAtributosQuarto)

    #Juntando tudo em uma única lista com nomes de hotel, atributos do ho
    linhasDataFrame = []
    numeroLinhas = len(listaNomesHotel)
    i=0
    while i < numeroLinhas:
        linhasDataFrame.append([listaNomesHotel[i]] + listasBinariasAtributosHoteis[i] + listasBinariasAtributosQuartos[i])
        i += 1

    cabecalhoDataFrame = ['Nome'] + comodidadesHotel + comodidadesQuarto

    #Criando um dataframe com todos os atributos juntos (nome do hotel + comods hotel + comods quarto)
    dtf = pd.DataFrame(linhasDataFrame,columns=cabecalhoDataFrame,dtype=str)
    
    #Acrescentando aos das páginas anteriores
    if cont == 1:
        pdAppended = dtf
    else:
        pdAppended = pd.concat([pdAppended,dtf],ignore_index=True)
    cont += 1

# pdAppended.to_csv('saidaDataScrapingInfo.csv',sep=';',encoding='utf-8')

# pdAppended
# pdGeralDataScraping = pd.concat([pdAppended, pdGeralDataScraping], axis=1)

pdGeralDataScraping["Nome"] = pdGeralDataScraping.Nome.astype("category")
pdAppended["Nome"] = pdAppended.Nome.astype("category")

pdGeral = pd.merge(pdGeralDataScraping, pdAppended, on="Nome")

# pdGeral.to_csv('saida3.csv',sep=';',encoding='utf-8')

pdGeral

,Nome,localizacao,quartos,servico,limpeza,custo-beneficio,conforto,facilidades,predio,cafe_da_manha,...,Geladeira,Internet,Mesa de trabalho,Microondas,Secador,Telefone,TV a cabo,TV LCD,Varanda / Terraço,Wi-Fi grátis nos quartos
0,Radisson Recife,92,84,79,85.0,78,85,83,80,Não avaliadoNão disponível,...,0,1,1,0,1,1,1,1,0,1
1,Golden Tulip Recife Boa Viagem,89,80,84,81.0,76,81,80,75,Não avaliadoNão disponível,...,1,1,1,0,1,1,1,0,0,0
2,Mar Hotel Conventions,86,82,79,84.0,77,83,83,79,70,...,0,1,1,0,1,1,1,0,0,1
3,Best Western Hotel Manibu Recife,84,77,82,86.0,77,82,82,82,75,...,0,1,1,0,1,1,1,1,0,1
4,Transamérica Prestige Beach Class International,89,84,88,88.0,79,89,85,85,Não avaliadoNão disponível,...,0,1,1,0,1,0,1,0,0,1
5,Beach Class Conselheiro By Nobile,Não avaliadoNão disponível,Não avaliadoNão disponível,83,87.0,Não avaliadoNão disponível,83,Não avaliadoNão disponível,81,Não avaliadoNão disponível,...,1,1,1,1,1,0,1,1,1,0
6,Hotel Atlante Plaza,91,81,84,85.0,77,84,81,78,78,...,0,1,1,0,1,1,1,0,0,1
7,Nobile Beach Class Executive,88,83,84,87.0,79,86,84,81,72,...,0,1,1,0,1,1,1,0,0,1
8,Hotel Duble,84,72,78,77.0,75,72,78,70,Não avaliadoNão disponível,...,0,1,1,0,1,1,1,1,0,1
9,Parkhotel,89,76,75,81.0,75,77,79,72,78,...,1,1,1,0,0,1,1,0,0,1


In [48]:
pdGeral

,Academia_x,Acesso Wi-Fi nos quartos_x,Acesso direto à praia_x,Animais permitidos_x,Ar condicionado_x,Banheiro com banheira_x,Banheiro com ducha_x,Bar na beira da Piscina_x,Bar no hotel_x,Cabeleireiro_x,...,Geladeira,Internet,Mesa de trabalho,Microondas,Secador,Telefone,TV a cabo,TV LCD,Varanda / Terraço,Wi-Fi grátis nos quartos
0,1,1,0,0,1,0,0,0,1,0,...,0,1,1,0,1,1,1,1,0,1
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,1,1,0,1,1,1,1,0,1
2,1,1,0,1,1,0,1,1,1,1,...,1,1,1,0,1,1,1,0,0,0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,1,0,1,1,1,0,0,0
4,1,1,0,1,1,1,1,1,1,1,...,0,1,1,0,1,1,1,0,0,1
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,1,1,0,1,1,1,0,0,1
6,1,1,0,0,1,1,1,0,1,0,...,0,1,1,0,1,1,1,1,0,1
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,1,1,0,1,1,1,1,0,1
8,1,1,0,1,1,0,1,1,1,1,...,0,1,1,0,1,1,1,0,0,1
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,1,1,0,1,1,1,0,0,1
